## ANN Classification End-to-End project

In [1]:
import numpy as numpy
import pandas as pd
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

#### Reading the dataset

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Data cleaning

In [4]:
# Dropping the first 3 columns as they are not important
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [5]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [6]:
# Missing/NaN values
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

#### Feature engineering

In [7]:
# Numerical and Ctaegorical features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
# Encoding the categorical features: Geography, Gender
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [9]:
df['Gender'].unique()

array([0, 1])

In [10]:
# Applying OneHotEncoding to Geography column
one_hot_encoder_geography = OneHotEncoder(sparse_output=False)
geo_encoder = one_hot_encoder_geography.fit_transform(df[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
geo_columns = one_hot_encoder_geography.get_feature_names_out()
geo_columns

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
df_geography = pd.DataFrame(data=geo_encoder, columns=geo_columns)
df_geography.head(2)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0


In [13]:
# Combining OneHotEncoded columns with the actual dataframe
df = pd.concat([df_geography, df], axis=1)
df.head(2)

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,0.0,0.0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,0.0,0.0,1.0,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [14]:
# Dropping 'Geography' column
df.drop(columns=['Geography'], axis=1, inplace=True)
df.head(2)

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,0.0,0.0,619,0,42,2,0.00,1,1,1,101348.88,1
1,0.0,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Geography_France   10000 non-null  float64
 1   Geography_Germany  10000 non-null  float64
 2   Geography_Spain    10000 non-null  float64
 3   CreditScore        10000 non-null  int64  
 4   Gender             10000 non-null  int32  
 5   Age                10000 non-null  int64  
 6   Tenure             10000 non-null  int64  
 7   Balance            10000 non-null  float64
 8   NumOfProducts      10000 non-null  int64  
 9   HasCrCard          10000 non-null  int64  
 10  IsActiveMember     10000 non-null  int64  
 11  EstimatedSalary    10000 non-null  float64
 12  Exited             10000 non-null  int64  
dtypes: float64(5), int32(1), int64(7)
memory usage: 976.7 KB


In [16]:
# Saving the preprocessor files in the pkl format
def save_object(file_path, preprocessor_object):
    with open(file_path, 'wb') as file:
        pickle.dump(preprocessor_object, file)

In [17]:
# Saving LabelEncoder
save_object('label_encoder_gender.pkl', label_encoder_gender)

In [18]:
# Saving OneHotEncoder
save_object('onehot_encoder_geography.pkl', one_hot_encoder_geography)

In [19]:
# Independent and dependent features
X = df.drop(columns=['Exited'], axis=1)
y = df['Exited']

In [20]:
# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [21]:
# Standard Scaling on numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
# Saving the StandardScaler
save_object('standard_scaler.pkl', scaler)

#### ANN Implementation

In [23]:
import tensorflow as tf

NOTE: In tensorflow, ANN is known as Sequential Model

In [24]:
# Importing sequential model
from tensorflow.keras.models import Sequential
# Importing Dense for creating Hidden neurons
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [25]:
# Building the Sequential model (ANN)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),   # Hidden Layer 1 which is connected with Input layer
    Dense(32, activation='relu'),   # Hidden Layer 2
    Dense(1, activation='sigmoid')   # Output layer
])

In [26]:
# Information about the model (model_structure, model_params etc)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Defining the optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# Defining the loss function
loss = tf.keras.losses.BinaryCrossentropy()

In [28]:
# For forward and backward propagation, we need to compile the model
model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [29]:
# Setting up Tensorboard for logging purpose
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [30]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [31]:
# Model Training
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
          callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 6s 10ms/step - loss: 387.5936 - accuracy: 0.6834 - val_loss: 26.8204 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 47.5900 - accuracy: 0.6835 - val_loss: 13.2445 - val_accuracy: 0.2610
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 12.2469 - accuracy: 0.6731 - val_loss: 4.0378 - val_accuracy: 0.6135
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 2.9165 - accuracy: 0.6930 - val_loss: 1.4299 - val_accuracy: 0.8030
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 1.2564 - accuracy: 0.7221 - val_loss: 0.6191 - val_accuracy: 0.8025
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6731 - accuracy: 0.7470 - val_loss: 0.5288 - val_accuracy: 0.7820
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5377 - accuracy: 0.7879 - val_loss: 0.5287 - val_accurac

In [32]:
# Saving the ANN model
model.save('ann_model.h5')  # h5 file which is compatible with Keras

c:\Users\Swastika\OneDrive\Documents\Udemy Courses\Data analyst bootcamp\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
# Load Tensorboard extension
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 21264), started 19:03:28 ago. (Use '!kill 21264' to kill it.)